In [3]:
from modules.dataset.entities import Entities
from modules.dataset.tweets import Tweets
from modules.network import HashNet

import networkx as nx
import numpy as np
import community

## Create network

In [4]:
TWEETS_DB_PATH = 'data/db/tweets.json'  # Path to tweets dataset
HASHTAGS_DB_PATH = 'data/db/hashtags.json' # Path to hashtags dataset

In [5]:
# Define networks container, indexed by periods
networks = {
    2017: None,
    2018: None,
    2019: None
}

In [6]:
# Import tweets dataset
tweets = Tweets()
tweets.from_json(TWEETS_DB_PATH)
tweets.df.head()

,tweet_id,tweet_date,tweet_text
0,836950901495631872,2017-03-01 14:46:59,TT SINGAPORE 22:46\n1.Hong Kong\n2.#JointAddre...
1,836950882528989184,2017-03-01 14:46:54,Letting #snapchat prepare me for the day's uns...
2,836950869639835649,2017-03-01 14:46:51,"""The bill would require the state to get all o..."
3,836950847380668416,2017-03-01 14:46:46,Style-Lead don't Follow #recycledfashion https...
4,836950839101116421,2017-03-01 14:46:44,‘Shell knew’: oil giant's 1991 film warned of ...


In [7]:
hashtags = Entities()
hashtags.from_json(HASHTAGS_DB_PATH)
hashtags.df.head()

,tweet_id,entity_index,entity_text,entity_tag,entity_conf
0,1101574442575167489,0,#Humans,#,0.6243
1,1101574442575167489,7,#climatechange,^,0.4925
2,1101574446341607424,27,#ClimateChange,#,0.6968
3,1101574446341607424,36,#ActOnClimate,#,0.7929
4,1101574446341607424,37,#climate,#,0.9559


In [8]:
# Define the list of hashtags used as search seeds
seed_list = ["#climatechange", "#climate", "#sdgs", "#sustainability", "#environment", "#globalwarming"]

for period in networks.keys():
    # Subset tweets for current period
    curr_tweets = Tweets()
    curr_tweets.df = tweets.df.loc[tweets.df.tweet_date.dt.year == period].copy()
    # Get ids of tweets for current period
    curr_tweets_id = curr_tweets.df.tweet_id.unique()
    # Subset hashtags and words associated to current tweets
    curr_hashtags = Entities()
    curr_hashtags.df = hashtags.df.loc[hashtags.df.tweet_id.isin(curr_tweets_id)].copy()
    # Lower hashtags
    curr_hashtags.df["entity_text"] = curr_hashtags.df["entity_text"].str.lower()
    # Remove seeds from hashtag dataset
    curr_hashtags.df = curr_hashtags.df.loc[~curr_hashtags.df.entity_text.isin(seed_list)]
    #curr_words.df = curr_words.df.loc[~curr_words.df.entity_text.isin(seed_list)]
    # Generate a dictionary containing words and hashtags networks for each period
    networks[period] = HashNet.from_entities(curr_hashtags)
    
    del curr_hashtags, curr_tweets

In [9]:
for period in networks.keys():
    net_type = "hashtags"
    # Get the list of connected components sorted by size
    cc = networks[period].get_connected_components()
    # If there is only one cc
    if len(cc) == 1:
        print("{} {} network is connected".format(period, net_type))
    else:
        # Compute the ratio between the size of the largest cc and the sum of all the cc sizes
        gc_ratio = int(100*cc[0]["size"]/sum([cc[i]["size"] for i in range(len(cc))]))
        # Print results
        print("{} {} network consists in {} connected components".format(period, net_type, len(cc)))
        print("The largest cc corresponds to {}% of total".format(gc_ratio))

2017 hashtags network consists in 218 connected components
The largest cc corresponds to 77% of total
2018 hashtags network consists in 206 connected components
The largest cc corresponds to 86% of total
2019 hashtags network consists in 185 connected components
The largest cc corresponds to 88% of total


In [10]:
# Keep only the largest connected components
for period in networks.keys():
    # Get the largest connected component
    lcc = networks[period].get_connected_components()[0]["component"]
    # Project the network on the lcc
    networks[period] = networks[period].project_component(lcc)

## Community detection

In [11]:
def getCommunitiesLouvain(network, resolution=1.0, threshold = 100):
    """
    Filter communities with less than threshold hashtags
    return: 
        comm: map community_id -> list of hashtags
        partitions: map hashtag -> community_id
    """ 
    # compute best partitions (fixed random state for reproducibility)
    partition = community.best_partition(graph=network, weight='weight', resolution=resolution, random_state=100)
    size = float(len(set(partition.values())))
    print('There are {} communities'.format(size))
    
    # partition is a dictionary in the form {'hashtag':community_id, ...}
    # we want ot transform it in the form {'community_id':[hashtag1, hashtag2, ...]}
    communities = {}
    for p in partition:
        if partition[p] in communities:
            communities[partition[p]].append(p)
        else:
            communities[partition[p]] = [p]

    # delete small communities (size(community)<threshold)
    communities = {k:v for k, v in communities.items() if len(v)>threshold}
    print('-> {} communities remaining after filtering'.format(len(communities)))
    return communities, partition

In [13]:
communities = {
    2017: None,
    2018: None,
    2019: None
}

parameters = {
    2017: {'resolution':0.9, 'threshold':70},
    2018: {'resolution':0.9, 'threshold':100},
    2019: {'resolution':0.9, 'threshold':100},
}

for period in networks.keys():
    print("--- Year: {} ---".format(period))
    comm, partitions = getCommunitiesLouvain(
        networks[period].net,
        resolution=parameters[period]['resolution'],
        threshold=parameters[period]['threshold']
    )
    communities[period] = {
        'communities': comm,
        'partitions': partitions
    }
    print()

--- Year: 2017 ---
There are 33.0 communities
-> 13 communities remaining after filtering

--- Year: 2018 ---
There are 35.0 communities
-> 20 communities remaining after filtering

--- Year: 2019 ---
There are 35.0 communities
-> 18 communities remaining after filtering



In [14]:
def sort_dict(d, descending=True):
    """
    Sort a dictionary based on items
    """
    return {k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}

def printCommunitiElemnts(
    network, communities, num_communities=5,
    community_id=None, metric_name='centrality',
    showTopK=5, print_metric=True
):
    """
    Print most important nodes based on certain metric
    """
    if metric_name=='centrality':
        centrality = nx.degree_centrality(network)
        metric = centrality
        
    if metric_name=='degree':
        degree = nx.degree(network, weight='weight')
        metric = degree
        
    # find most relevant (based on metric) terms for each community
    if community_id == None:
        counter = 1
        for k,v in communities.items():
            print('---Community: {}, Size: {}---'.format(k, len(v)))
            tag_dict = {tag: metric[tag] for tag in v}
            tag_dict = sort_dict(tag_dict)
            tag_list = list(tag_dict.keys())
            for tag in tag_list[:showTopK]:
                if print_metric:
                    print(tag, ',{}: {:.4f}'.format(metric_name, metric[tag]))
                else:
                    print(tag)
            print()
            ## Show only num_communities
            if counter == num_communities:
                break
            counter += 1
    else:
        k = community_id
        if k not in communities:
            print("Community {} has been discarded (too small)".format(k))
            return
        v = communities[community_id]
        print('---Community: {}---'.format(k))
        tag_dict = {tag: metric[tag] for tag in v}
        tag_dict = sort_dict(tag_dict)
        tag_list = list(tag_dict.keys())
        for tag in tag_list[:showTopK]:
            if print_metric:
                print(tag, ',{}: {:.4f}'.format(metric_name, metric[tag]))
            else:
                print(tag)
        print()

In [15]:
year = 2019

printCommunitiElemnts(
    networks[year].net, communities[year]['communities'],
    num_communities=10, metric_name='centrality',
    print_metric=False, showTopK=7, community_id=None
)

---Community: 0, Size: 182---
#un
#sustainabledevelopment
#peace
#sdg
#nigeria
#unitednations
#2030agenda

---Community: 1, Size: 350---
#climateaction
#fridaysforfuture
#climatecrisis
#extinctionrebellion
#climateactionnow
#climateemergency
#schoolstrike4climate

---Community: 2, Size: 269---
#nature
#pollution
#conservation
#usa
#savetheplanet
#india
#trees

---Community: 3, Size: 313---
#innovation
#ai
#iot
#news
#circulareconomy
#plastic
#plasticfree

---Community: 4, Size: 269---
#energy
#climatestrike
#greennewdeal
#renewables
#solar
#renewableenergy
#actonclimate

---Community: 7, Size: 150---
#agriculture
#research
#environmental
#deforestation
#food
#farmers
#foodsecurity

---Community: 8, Size: 145---
#africa
#love
#art
#development
#tourism
#music
#cycloneidai

---Community: 9, Size: 157---
#green
#arctic
#zerowaste
#earthday
#adventure
#indigenous
#animals

---Community: 10, Size: 145---
#science
#climatechangeisreal
#resist
#scicomm
#gender
#migration
#fracking

---Communi

In [17]:
year = 2018

printCommunitiElemnts(
    networks[year].net, communities[year]['communities'],
    num_communities=10, metric_name='centrality',
    print_metric=False, showTopK=7, community_id=None
)

---Community: 0, Size: 156---
#weather
#iot
#engineering
#smartcities
#jobs
#people
#efficiency

---Community: 1, Size: 212---
#globalgoals
#design
#architecture
#construction
#landscape
#organic
#building

---Community: 2, Size: 333---
#energy
#solar
#renewables
#climateaction
#renewableenergy
#africa
#cdnpoli

---Community: 3, Size: 315---
#nature
#wildlife
#animals
#oceans
#photography
#spring
#forest

---Community: 4, Size: 156---
#parisagreement
#auspol
#climatechangeisreal
#earthday
#theresistance
#coal
#qldpol

---Community: 7, Size: 210---
#csr
#youth
#climatejustice
#esg
#cop24
#social
#sdg5

---Community: 8, Size: 238---
#sustainable
#waste
#business
#reuse
#environmental
#recycling
#startup

---Community: 9, Size: 123---
#women
#iwd2018
#gender
#tshirt
#reading
#nzpol
#unitednations

---Community: 10, Size: 238---
#innovation
#science
#education
#technology
#ecology
#development
#economy

---Community: 11, Size: 276---
#water
#agriculture
#food
#worldwaterday
#biodiversity
#

In [18]:
year = 2017

printCommunitiElemnts(
    networks[year].net, communities[year]['communities'],
    num_communities=10, metric_name='centrality',
    print_metric=False, showTopK=7, community_id=None
)

---Community: 0, Size: 136---
#globalgoals
#education
#iwd2017
#agenda2030
#logic
#womensday
#beboldforchange

---Community: 3, Size: 135---
#resist
#humanrights
#theresistance
#maga
#women
#usa
#republicans

---Community: 4, Size: 78---
#science
#auspol
#arctic
#jobs
#qldpol
#sciencemarch
#syria

---Community: 5, Size: 148---
#energy
#cdnpoli
#renewables
#solar
#renewableenergy
#waterislife
#actonclimate

---Community: 6, Size: 84---
#recycle
#ecofriendly
#sustainabledevelopment
#foodwaste
#waste
#change
#zerohunger

---Community: 11, Size: 91---
#nature
#wildlife
#conservation
#biodiversity
#art
#disaster
#colombia

---Community: 12, Size: 90---
#trump
#epa
#coal
#gogreen
#earthday
#china
#earthhour2017

---Community: 16, Size: 85---
#health
#pollution
#airquality
#wellness
#airpollution
#cleanair
#healthyliving

---Community: 17, Size: 115---
#sustainable
#water
#earth
#food
#agriculture
#eco
#love

---Community: 18, Size: 84---
#innovation
#csr
#business
#impinv
#energyefficiency
#

In [19]:
# inspect community by id
year = 2019

printCommunitiElemnts(
    networks[year].net, communities[year]['communities'],
    num_communities=10, metric_name='centrality',
    print_metric=False, showTopK=10, community_id=1
)

---Community: 1---
#climateaction
#fridaysforfuture
#climatecrisis
#extinctionrebellion
#climateactionnow
#climateemergency
#schoolstrike4climate
#gretathunberg
#genderequality
#iwd2019

